# 📊 Analyse de performances de joueurs de football

# Analyse des performances de joueurs de football

Ce notebook analyse les performances de joueurs à partir des données issues de FBref, dans une logique de recrutement basé sur les statistiques avancées.

## 1. Chargement des données

In [ ]:
import pandas as pd

# Charger les données
file_path = "data/data2.0.xlsx"
df = pd.read_excel(file_path, sheet_name="Feuille 1")

# Supprimer la première ligne si elle est vide
df = df.dropna(how='all')

# Aperçu
df.head()


## 2. Nettoyage initial

On convertit les colonnes numériques, on traite les virgules comme séparateurs décimaux si besoin, et on sélectionne les colonnes clés.

In [ ]:
# Remplacer les virgules par des points et convertir les colonnes nécessaires
df['Min'] = df['Min'].astype(str).str.replace(',', '').astype(float)
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Gls'] = pd.to_numeric(df['Gls'], errors='coerce')
df['Ast'] = pd.to_numeric(df['Ast'], errors='coerce')
df['xG.1'] = pd.to_numeric(df['xG.1'], errors='coerce')
df['xAG.1'] = pd.to_numeric(df['xAG.1'], errors='coerce')
df['xG+xAG'] = pd.to_numeric(df['xG+xAG'], errors='coerce')


## 3. Analyse exploratoire

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Distribution des xG
sns.histplot(df['xG.1'].dropna(), kde=True)
plt.title('Distribution des xG')
plt.show()

# Corrélation entre xG, xAG, Age, Min
sns.pairplot(df[['xG.1', 'xAG.1', 'Age', 'Min']].dropna())
plt.show()


## 4. Régression linéaire : prédiction de la note (ex: G+A par 90 min)

In [ ]:
import statsmodels.api as sm

# Variable cible (ex: G+A.1)
df_model = df[['G+A.1', 'xG.1', 'xAG.1', 'Age', 'Min']].dropna()

X = df_model[['xG.1', 'xAG.1', 'Age', 'Min']]
X = sm.add_constant(X)
y = df_model['G+A.1']

model = sm.OLS(y, X).fit()
print(model.summary())


## 5. Visualisation des résidus

In [ ]:
# Prédictions vs Réel
df_model['predicted'] = model.predict(X)

plt.scatter(df_model['predicted'], y)
plt.xlabel("Valeurs prédites")
plt.ylabel("Valeurs observées")
plt.title("Prédictions vs Réel")
plt.axline((0, 0), slope=1, color='red')
plt.show()
